In [1]:
import sys
import tensorflow as tf
sys.path.append('../../generative-adversarial-nets/')
from ganetwork import GAN, CGAN
from sklearn import datasets
from sklearn.preprocessing import minmax_scale, LabelBinarizer

In [2]:
ds = datasets.load_iris()
X, y = minmax_scale(ds.data), LabelBinarizer().fit_transform(ds.target)

In [ ]:
discriminator_layers=[(4, None), (8, tf.nn.tanh), (3, tf.nn.softmax), (1, None)]
generator_layers=[(6, None), (5, tf.nn.tanh), (4, None)]
gan = GAN(discriminator_layers, generator_layers)
gan.train(X, nb_epoch=50, batch_size=5)

In [3]:
discriminator_layers=[(7, None), (8, tf.nn.tanh), (3, tf.nn.softmax), (1, None)]
generator_layers=[(9, None), (5, tf.nn.tanh), (4, None)]
cgan = CGAN(discriminator_layers, generator_layers)

In [4]:
n_classes = y.shape[1]
X_placeholder, discriminator_parameters = CGAN._initialize_model(cgan.discriminator_layers, n_classes)
Z_placeholder, generator_parameters = CGAN._initialize_model(cgan.generator_layers, n_classes)
y_placeholder = tf.placeholder(tf.float32, [None, n_classes])

In [5]:
generator_logit = CGAN._output_logit_tensor(tf.concat(axis=1, values=[Z_placeholder, y_placeholder]), cgan.generator_layers, generator_parameters)
discriminator_logit_real = CGAN._output_logit_tensor(tf.concat(axis=1, values=[X_placeholder, y_placeholder]), cgan.discriminator_layers, discriminator_parameters)
discriminator_logit_generated = CGAN._output_logit_tensor(tf.concat(axis=1, values=[tf.nn.sigmoid(generator_logit), y_placeholder]), cgan.discriminator_layers, discriminator_parameters)

In [6]:
discriminator_loss = CGAN._return_loss(discriminator_logit_real, True) + GAN._return_loss(discriminator_logit_generated, False)
generator_loss = CGAN._return_loss(discriminator_logit_generated, True)

In [7]:
discriminator_update_step = CGAN._update_model_parameters(cgan.discriminator_optimizer, discriminator_loss, discriminator_parameters)
generator_update_step = CGAN._update_model_parameters(cgan.generator_optimizer, generator_loss, generator_parameters)

In [8]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [11]:
batch_size = 5
n_batches = int(X.shape[0] / batch_size)
n_Z_features = cgan.generator_layers[0][0] - n_classes

In [12]:
n_Z_features

6